In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import joblib
import re
from PIL import Image
# import dlib
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import sklearn
from sklearn.model_selection import train_test_split
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [13]:
path = "D:/Down/inha_data (2)/inha_data"
os.chdir(path)

In [14]:
# dataset = joblib.load('C:/Users/pc/OneDrive/문서/카카오톡 받은 파일/dataset/dataset.pkl')
dataset= joblib.load("D:/Down/dataset_big3.pkl")

In [15]:
y = dataset['label']
X = pd.DataFrame(dataset['data'])
X.columns = list(range(256)) + ["cosine_left", "cosine_right", "l1_left", "l1_right", "l2_left", "l2_right", "inf_left", "inf_right"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size =.3, random_state = 0)

In [16]:
test_left = []
test_right = []
for i in os.listdir("test"):
    p1 = re.compile("left")
    if p1.findall(i) == ['left']:
        p2 = re.compile("[0-9]")
        test_left.append(int("".join(p2.findall(i))))
    else:
        p3 = re.compile("[0-9]")
        test_right.append(int("".join(p3.findall(i))))        

test_left.sort()
test_right.sort()
test_left = list(map( lambda x :"test/left_face_"+str(x)+".jpg", test_left))
test_right = list(map( lambda x :"test/right_face_"+ str(x)+".jpg", test_right))

In [18]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_neurons, n_layers,  **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation = 'relu', kernel_initializer = "he_normal") for _ in range(n_layers)]
    def call(self, inputs):
        Z = inputs
        Z_copy = Z[:]
        for layer in self.hidden:
            Z = layer(Z)
        return Z_copy + Z
    


class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, n_neurons1, n_neurons2, n_layer,**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(n_neurons1, activation = "relu", kernel_initializer="he_normal")
        self.hidden2 = tf.keras.layers.Dense(n_neurons2, activation = "relu", kernel_initializer = "he_normal")
        self.block1 = ResidualBlock(n_neurons1, n_layer)
        self.out = tf.keras.layers.Dense(output_dim, activation = 'sigmoid', name = 'output')
        
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(2):
            Z = self.block1(Z)
        Z =self.hidden2(Z)
        return self.out(Z)


In [31]:
def build_model(output_dim, n_neurons1, n_neurons2, n_layer):
    model = ResidualRegressor(output_dim, n_neurons1, n_neurons2, n_layer)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer)
    return model

In [20]:
param = {
    "output_dim" : [1],
    "n_neurons1" : [128, 64, 32],
    "n_neurons2" : [64, 32],
    "n_layer" : [3,2,1]
}

In [33]:
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model, output_dim =1, n_neurons1 = 64, n_neurons2 = 64, n_layer = 1)
rsc_reg = RandomizedSearchCV(keras_reg,param, n_iter=10, cv =5)

In [35]:
keras_reg.fit(X_train.iloc[:, :].values, y_train,epochs = 100, validation_data = (X_valid.iloc[:, :].values, y_valid),
          batch_size = 32, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Train on 1216227 samples, validate on 521241 samples
Epoch 1/100
1216227/1216227 [==============================] - 87s 71us/sample - loss: 0.1216 - val_loss: 0.1165
Epoch 2/100
1216227/1216227 [==============================] - 86s 71us/sample - loss: 0.1121 - val_loss: 0.1106
Epoch 3/100
1216227/1216227 [==============================] - 86s 71us/sample - loss: 0.1096 - val_loss: 0.1091
Epoch 4/100
1216227/1216227 [==============================] - 86s 70us/sample - loss: 0.1082 - val_loss: 0.1085
Epoch 5/100
1216227/1216227 [==============================] - 86s 70us/sample - loss: 0.1073 - val_loss: 0.1065
Epoch 6/100
1216227/1216227 [==============================] - 85s 70us/sample - loss: 0.1066 - val_loss: 0.1073
Epoch 7/100
1216227/1216227 [==============================] - 85s 70us/sample - loss: 0.1062 - val_loss: 0.1057
Epoch 8/100
1216227/1216227 [==============================] - 85s 70us/sample - loss: 0.1056 - val_loss: 0.1065
Epoch 9/100
1216227/1216227 [==============

In [406]:
rsc_reg.fit(X_train.iloc[:, :].values, y_train,epochs = 100, validation_data = (X_valid.iloc[:, :].values, y_valid),
          batch_size = 128,validation_batch_size=128, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
8688/8688 [==============================] - 12s 1ms/step - loss: 0.1240 - val_loss: 0.1161
Epoch 2/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1120 - val_loss: 0.1133
Epoch 3/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1092 - val_loss: 0.1083
Epoch 4/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1075 - val_loss: 0.1064
Epoch 5/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1063 - val_loss: 0.1052
Epoch 6/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1053 - val_loss: 0.1073
Epoch 7/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1045 - val_loss: 0.1070
Epoch 8/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1038 - val_loss: 0.1055
Epoch 9/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1033 - val_loss: 0.1057
Epoch 10/100
8688/8688 [==============================] - 9s 1ms/step 

8688/8688 [==============================] - 10s 1ms/step - loss: 0.1030 - val_loss: 0.1047
Epoch 12/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1025 - val_loss: 0.1050
Epoch 13/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1021 - val_loss: 0.1047
Epoch 14/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1019 - val_loss: 0.1053
Epoch 15/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1015 - val_loss: 0.1054
Epoch 16/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1013 - val_loss: 0.1054
Epoch 17/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1010 - val_loss: 0.1100
Epoch 18/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1009 - val_loss: 0.1066
Epoch 19/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1007 - val_loss: 0.1043
Epoch 20/100
8688/8688 [==============================] - 10s 1ms/st

Epoch 18/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0945 - val_loss: 0.1046
Epoch 19/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0940 - val_loss: 0.1035
Epoch 20/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0937 - val_loss: 0.1040
Epoch 21/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0933 - val_loss: 0.1034
Epoch 22/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0929 - val_loss: 0.1052
Epoch 23/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.0926 - val_loss: 0.1047
Epoch 24/100
8688/8688 [==============================] - 4s 465us/step - loss: 0.1067
Epoch 1/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1220 - val_loss: 0.1107
Epoch 2/100
8688/8688 [==============================] - 10s 1ms/step - loss: 0.1113 - val_loss: 0.1081
Epoch 3/100
8688/8688 [==============================] - 10s 1ms/step - los

8688/8688 [==============================] - 9s 997us/step - loss: 0.1026 - val_loss: 0.1057
Epoch 21/100
8688/8688 [==============================] - 8s 955us/step - loss: 0.1022 - val_loss: 0.1049
Epoch 22/100
8688/8688 [==============================] - 8s 962us/step - loss: 0.1022 - val_loss: 0.1046
Epoch 23/100
8688/8688 [==============================] - 8s 949us/step - loss: 0.1020 - val_loss: 0.1049
Epoch 24/100
8688/8688 [==============================] - 8s 958us/step - loss: 0.1020 - val_loss: 0.1044
Epoch 25/100
8688/8688 [==============================] - 8s 956us/step - loss: 0.1018 - val_loss: 0.1044
Epoch 26/100
8688/8688 [==============================] - 8s 966us/step - loss: 0.1018 - val_loss: 0.1051
Epoch 27/100
8688/8688 [==============================] - 8s 962us/step - loss: 0.1015 - val_loss: 0.1043
Epoch 28/100
8688/8688 [==============================] - 8s 964us/step - loss: 0.1015 - val_loss: 0.1044
Epoch 29/100
8688/8688 [==============================] - 8

Epoch 45/100
8688/8688 [==============================] - 9s 981us/step - loss: 0.1010 - val_loss: 0.1038
Epoch 46/100
8688/8688 [==============================] - 8s 975us/step - loss: 0.1009 - val_loss: 0.1036
Epoch 47/100
8688/8688 [==============================] - 8s 975us/step - loss: 0.1010 - val_loss: 0.1032
Epoch 48/100
8688/8688 [==============================] - 8s 978us/step - loss: 0.1009 - val_loss: 0.1031
Epoch 49/100
8688/8688 [==============================] - 8s 970us/step - loss: 0.1009 - val_loss: 0.1038
Epoch 50/100
8688/8688 [==============================] - 8s 967us/step - loss: 0.1009 - val_loss: 0.1066
Epoch 51/100
8688/8688 [==============================] - 9s 979us/step - loss: 0.1011 - val_loss: 0.1036
Epoch 52/100
8688/8688 [==============================] - 8s 976us/step - loss: 0.1008 - val_loss: 0.1050
Epoch 53/100
8688/8688 [==============================] - 8s 976us/step - loss: 0.1009 - val_loss: 0.1065
Epoch 54/100
8688/8688 [======================

...

Epoch 40/100
8688/8688 [==============================] - 8s 938us/step - loss: 0.1020 - val_loss: 0.1045
Epoch 41/100
8688/8688 [==============================] - 8s 935us/step - loss: 0.1022 - val_loss: 0.1059
Epoch 42/100
8688/8688 [==============================] - 8s 936us/step - loss: 0.1020 - val_loss: 0.1055
Epoch 43/100
8688/8688 [==============================] - 8s 937us/step - loss: 0.1019 - val_loss: 0.1060
Epoch 44/100
8688/8688 [==============================] - 8s 941us/step - loss: 0.1020 - val_loss: 0.1062
Epoch 45/100
8688/8688 [==============================] - 8s 926us/step - loss: 0.1017 - val_loss: 0.1041
Epoch 46/100
8688/8688 [==============================] - 8s 936us/step - loss: 0.1018 - val_loss: 0.1044
Epoch 47/100
8688/8688 [==============================] - 8s 936us/step - loss: 0.1020 - val_loss: 0.1062
Epoch 48/100
8688/8688 [==============================] - 8s 935us/step - loss: 0.1017 - val_loss: 0.1054
Epoch 49/100
8688/8688 [======================

Epoch 12/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1014 - val_loss: 0.1050
Epoch 13/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1010 - val_loss: 0.1042
Epoch 14/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1005 - val_loss: 0.1068
Epoch 15/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.1003 - val_loss: 0.1032
Epoch 16/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.0998 - val_loss: 0.1042
Epoch 17/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.0996 - val_loss: 0.1035
Epoch 18/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.0993 - val_loss: 0.1031
Epoch 19/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.0990 - val_loss: 0.1042
Epoch 20/100
8688/8688 [==============================] - 9s 1ms/step - loss: 0.0988 - val_loss: 0.1044
Epoch 21/100
8688/8688 [==============================] - 10s 1m

Epoch 16/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1019 - val_loss: 0.1037
Epoch 17/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1012 - val_loss: 0.1036
Epoch 18/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1017 - val_loss: 0.1056
Epoch 19/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1009 - val_loss: 0.1038
Epoch 20/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1010 - val_loss: 0.1044
Epoch 21/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1004 - val_loss: 0.1040
Epoch 22/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1002 - val_loss: 0.1039
Epoch 23/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1004 - val_loss: 0.1033
Epoch 24/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.0999 - val_loss: 0.1035
Epoch 25/100
8688/8688 [==============================]

Epoch 9/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1040 - val_loss: 0.1066
Epoch 10/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1034 - val_loss: 0.1049
Epoch 11/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1029 - val_loss: 0.1047
Epoch 12/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1027 - val_loss: 0.1039
Epoch 13/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1022 - val_loss: 0.1046
Epoch 14/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1018 - val_loss: 0.1052
Epoch 15/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1017 - val_loss: 0.1046
Epoch 16/100
8688/8688 [==============================] - 12s 1ms/step - loss: 0.1012 - val_loss: 0.1035
Epoch 17/100
8688/8688 [==============================] - 11s 1ms/step - loss: 0.1009 - val_loss: 0.1066
Epoch 18/100
8688/8688 [==============================] 

RandomizedSearchCV(cv=5,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000020E6BC73088>,
                   param_distributions={'n_layer': [3, 2, 1],
                                        'n_neurons1': [128, 64, 32],
                                        'n_neurons2': [64, 32],
                                        'output_dim': [1]})

In [41]:
keras_reg.model.save("keras_reg")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: keras_reg\assets


In [37]:
yp = keras_reg.predict(X_valid.iloc[:, :].values)
m = tf.keras.metrics.AUC()
m.update_state(y_valid, yp)
m.result().numpy()

0.9923962

In [410]:
testdat = joblib.load('C:/Users/pc/OneDrive/문서/카카오톡 받은 파일/dataset_test.pkl')
veri_possible = testdat['idx']
testdat = testdat['data']

In [411]:
predict_y = model.predict(testdat[:, :])

In [412]:
submission = pd.read_csv("sample_submission.csv")
submission.answer = .5
submission.answer[veri_possible] = predict_y.reshape((-1,))
submission.to_csv("residualblock64.csv",index=False)

199103